In [ ]:
import requests
import pandas as pd
import json
from datetime import datetime
import logging

# Configure the logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

force_none_proxies = {
    "http": None,
    "https": None,
}

import hashlib


def get_md5_encoded_string(phone_number, date, word):
    # Concatenate the input values
    input_string = f"{phone_number}{date}{word}"

    # Encode the string to bytes
    input_bytes = input_string.encode("utf-8")

    # Generate the MD5 hash
    md5_hash = hashlib.md5(input_bytes)

    # Convert the hash to its hexadecimal representation
    md5_hex = md5_hash.hexdigest()

    return md5_hex


# Example usage
phone_number = "18618107293"
date = datetime.now().strftime("%Y%m%d")
word = "login"
md5_encoded_string = get_md5_encoded_string(phone_number, date, word)
logging.info(md5_encoded_string)


def get_token_for_env(env="https://qah5.summerfarm.net"):
    url = f"{env}/openid?phone={phone_number}&sign={md5_encoded_string}"
    logging.info(url)
    token = requests.get(url=url, timeout=12000, proxies=force_none_proxies)
    logging.info(token.status_code, f"\ntext:{token.text}")
    try:
        token = json.loads(token.text)
        logging.info(token)
        return {"env": f"{env}", "token": token["data"]["token"]}
    except Exception as e:
        logging.error(f"获取Token失败:{url}, {token}, {e}")
        raise e

In [ ]:
from sls_client import get_sls_data_by_query
from datetime import datetime, timedelta

to_time = datetime.now()- timedelta(days=0)
from_time = datetime.now() - timedelta(hours=1)

query = """inboundFlag: POST\:/price/query/take-actual-price and "requestBody = "| 
select json_extract(split(msg,'requestBody = ')[2], '$[0]') as orderNow 
from log group by 1 order by count(1) desc limit 200"""
sku_object_list_from_sls = get_sls_data_by_query(
    from_time=from_time,
    to_time=to_time,
    query=query,
    project="k8s-log-c7d28cba17d0a416ca4f52459592b8d38",
    logstore="prod-mall-stdout-log",
)

sku_object_list_from_sls.head(2)


mall_token_list=get_sls_data_by_query(from_time=from_time, to_time=to_time, query="""
token and "杭州"|select token from log group by 1 having token like 'mall__%' limit 100
""")

mall_token_list.head(2)

In [ ]:
sku_object_list = []
user_list=[]
for index, row in sku_object_list_from_sls.iterrows():
    sku_object_list.append(json.loads(row["orderNow"]))

for index, row in mall_token_list.iterrows():
    user_list.append(row['token'])

print(len(sku_object_list), sku_object_list[0])

## 开始做验证

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import concurrent
import pandas as pd
import re
import random


def get_take_price_result(token="", order_now_obj={}) -> [int, str]:
    url = f"https://test619.summerfarm.net/price/query/take-actual-price"
    logging.info(f"making requests to {url}\nSKU:{order_now_obj}, token:{token}")
    started_at = int(datetime.now().timestamp() * 1000)
    try:
        data = requests.post(
            url=url,
            json=order_now_obj,
            headers={"token": token},
            proxies=force_none_proxies,
            timeout=(1, 1),
        ).json()
        if data is not None and "data" in data:
            data = data.get("data")
        else:
            logging.error(
                f"获取到了错误的结果token:{token}, data:{data}, sku:{order_now_obj}"
            )
            data = [data]
        # logging.info(f"env:{env_object}\ndata:{data}")
        finished_at = int(datetime.now().timestamp() * 1000)
        rt = finished_at - started_at
        # logging.info(f"RT:{rt}ms")
        return rt, data
    except Exception as e:
        logging.error(f"发生了异常:{e}")
        return 2000, [f"异常-可能是http超时了:{e}"]


def get_random_6elements(num=1):
    # Get a random sample of n elements
    return random.sample(sku_object_list, num)


df = []

five_minutes_ahead = datetime.now() + timedelta(minutes=1)


def process_sku():
    global five_minutes_ahead
    global df
    while datetime.now() < five_minutes_ahead:
        sku_info = random.sample(sku_object_list, 1)[0]
        token = random.sample(user_list, 1)[0]
        df_object = {}
        rt, data = get_take_price_result(token=token, order_now_obj=sku_info)
        logging.info(f"sku_info:{sku_info}, data:{data}")
        df_object["requested_sku_object"] = sku_info
        df_object["rt"] = rt
        df_object["data"] = data
        df.append(df_object)
    logging.info("stoped:")


max_workers = 20

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Submit tasks to the executor
    futures = [executor.submit(process_sku) for idex in range(0, max_workers)]
    concurrent.futures.wait(futures)

df = pd.DataFrame(df)
df.head(1)

In [ ]:
print(df.describe(),'\n\n', df.quantile([0.5,0.75,0.90,0.99,0.999,0.9999]))

In [ ]:
df.head(10)